In [1]:
import plotly.express as px
import pandas as pd
import json

from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import numpy as np
import sys
import os

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.config import config
from src.data.etl import load_data
from src.model.bert import TextEmbedder, DimensionalityReducer, ClusteringModel, CTFIDFVectorizer
from src.model.visualize import TopicVisualizer
from src.model.eval import TopicModelEvaluator
from src.model.label import LLMTopicNamer, KeyBERTInspired

from dotenv import load_dotenv

# os.environ['HF_HUB_OFFLINE'] = '1' # Force offline mode for HuggingFace
# os.environ['TRANSFORMERS_OFFLINE'] = '1' # Force offline mode for Transformers
load_dotenv()

/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from huggingface_hub import scan_cache_dir

cache_info = scan_cache_dir()
print("Cached models:")
for repo in cache_info.repos:
    print(f"  - {repo.repo_id} ({repo.size_on_disk_str})")
    
cache_dir = Path.home() / ".cache" / "huggingface" / "hub"
print(f"HuggingFace cache directory: {cache_dir}")
print(f"Exists: {cache_dir.exists()}")

Cached models:
  - Qwen/Qwen3-4B (8.1G)
  - BAAI/bge-small-en-v1.5 (134.5M)
  - Qwen/Qwen3-Embedding-0.6B (1.2G)
  - unsloth/Qwen3-8B-GGUF (5.0G)
  - cross-encoder/ms-marco-MiniLM-L-6-v2 (91.8M)
  - FacebookAI/roberta-base (2.7M)
  - Qwen/Qwen3-8B-AWQ (1.0K)
  - mlx-community/Qwen3-8B-4bit (939.0)
  - sentence-transformers/all-MiniLM-L6-v2 (91.6M)
  - BAAI/bge-large-en-v1.5 (1.3G)
  - Qwen/Qwen3-8B (16.4G)
HuggingFace cache directory: /Users/Z00GK5Z/.cache/huggingface/hub
Exists: True


In [3]:
print(config)

Settings Configuration:
  BASE_PATH: /Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling
  DATA_FILE_PATH: /Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/data/raw/data.json
  SEED: 23
  LLM_CONFIG: {'model_name': 'Qwen/Qwen3-4B', 'max_new_tokens': 80, 'temperature': 0.1, 'use_case': 'customer_reviews'}
  LOG_CONFIG: {'version': 1, 'disable_existing_loggers': False, 'formatters': {'detailed': {'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s', 'datefmt': '%Y-%m-%d %H:%M:%S'}, 'simple': {'format': '%(levelname)s - %(message)s'}}, 'handlers': {'console': {'class': 'logging.StreamHandler', 'level': 'INFO', 'formatter': 'simple', 'stream': 'ext://sys.stdout'}, 'file': {'class': 'logging.handlers.RotatingFileHandler', 'level': 'DEBUG', 'formatter': 'detailed', 'filename': '/Users/Z00GK5Z/Documents/Workspace/Project/topic_modeling/output/topic_modeling.log', 'maxBytes': 10485760, 'backupCount': 5, 'encoding': 'utf-8'}}, 'root': {'level': 'INFO', 'handlers': ['c

In [4]:
data = load_data()

data.columns = data.columns.str.strip().str.lower().str.replace(" ", "_")

print(data.columns)
data.head()

Index(['reviewerid', 'asin', 'reviewername', 'helpful', 'reviewtext',
       'overall', 'summary', 'unixreviewtime', 'reviewtime'],
      dtype='object')


,reviewerid,asin,reviewername,helpful,reviewtext,overall,summary,unixreviewtime,reviewtime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
item_counts = data.groupby("asin", dropna=False).size().sort_values(ascending=False)
top_asins = item_counts[(item_counts >= 20) & (item_counts <= 50)].index.tolist()

print(f"Number of ASINs with 20-50 reviews: {len(top_asins)}")
print(item_counts[(item_counts >= 20) & (item_counts <= 50)].head(5))


data = data[data['asin'].isin(top_asins)]
print(f"\nFiltered data shape: {data.shape}")

Number of ASINs with 20-50 reviews: 3
asin
B008BA33Q4    50
B009AWT01Y    50
B00AANQLRI    50
B00B78MT44    50
B0078WL4YO    50
dtype: int64

Filtered data shape: (150, 9)


In [6]:

def helpful_ratio(x):
    if isinstance(x, (list, tuple)) and len(x) >= 2:
        up, total = x[0], x[1]
        return up / max(total, 1)
    return 0.0

data["helpful_ratio"] = data["helpful"].apply(helpful_ratio)
data["help_bucket"] = pd.cut(
    data["helpful_ratio"],
    bins=[ 0.0, 0.25, 0.6, 1.0],
    labels=[ "HELP_LOW", "HELP_MED", "HELP_HIGH"],
).astype(str)

data["rating_bucket"] = data["overall"].fillna(0).astype(int).clip(0, 5).map(lambda r: f"RATING_{r}")


In [7]:
def run_topic_modeling_per_item(group, value, embedder, llm_namer, config):
    """
    Run topic modeling for a single item group
    
    Returns:
        List of dicts with dpci, topic, and reason
    """
    # Create documents from the group
    docs = []
    for _, row in group.iterrows():
        review = row["reviewtext"]
        meta_info = f" Rating: {row['overall']}; Helpfulness: {row['helpful_ratio']:.2f}; "
        doc = f" {review}"
        if doc:
            docs.append(doc)
    
    if len(docs) < 5:  # Skip if too few documents
        return []
    
    # Embed documents
    embeddings = embedder.embed(docs)
    
    # Reduce dimensions
    reducer = DimensionalityReducer(**config.dr_config)
    reduced_embeddings = reducer.fit_transform(embeddings)
    
    # Cluster
    cluster_model = ClusteringModel(**config.clustering_config)
    cluster_labels = cluster_model.fit_predict(reduced_embeddings)
    
    # Group docs by topic
    docs_per_topic = {}
    for doc, label in zip(docs, cluster_labels):
        if label == -1:
            continue
        if label not in docs_per_topic:
            docs_per_topic[label] = []
        docs_per_topic[label].append(doc)
    
    topic_docs = []
    topic_ids = []
    for topic_id in sorted(docs_per_topic.keys()):
        topic_docs.append(' '.join(docs_per_topic[topic_id]))
        topic_ids.append(topic_id)
    
    if not topic_docs:
        return []
    
    # Extract keywords with C-TF-IDF
    ctfidf_vectorizer = CTFIDFVectorizer(**config.c_tfidf_config)
    c_tfidf_matrix, vocab = ctfidf_vectorizer.fit_transform(topic_docs)
    
    top_n_words = 15
    
    
    keybert_refiner = KeyBERTInspired(
        embedder=embedder,
        top_n_words=top_n_words,
        nr_candidate_words=40  # Re-rank top 40 c-TF-IDF words
    )
    refined_topic_words = keybert_refiner.refine_topics(
        documents=docs,
        labels=cluster_labels,
        embeddings=embeddings,
        c_tf_idf=c_tfidf_matrix,
        vocab=vocab
    )
   
    # Generate topic names using LLM
    topic_names = llm_namer.name_topics(refined_topic_words, top_n_words=top_n_words)
    
    # Build results with topic names
    results = []
    for topic_id in topic_ids:
        results.append({
            'item': value,
            'topic': topic_names.get(topic_id, {"name":f"Topic {topic_id}"})['name'],
            'reason': topic_names.get(topic_id, {"reasoning":""})['reasoning'],
            'documents': docs_per_topic[topic_id],  
            'num_documents': len(docs_per_topic[topic_id])
        })
    
    return results


all_results = []
embedder = TextEmbedder(**config.embedding_model_config)
llm_namer = LLMTopicNamer() 
# ✅ Cache the LLM here
print("✅ Models initialized and cached!")

for item, group in data.groupby('asin', dropna=False):
    print(f"Processing ASIN: {item} ({len(group)} records)")
    results = run_topic_modeling_per_item(group, item, embedder, llm_namer, config)
    all_results.extend(results)

results_df = pd.DataFrame(all_results)
print(f"\nTotal topics found: {len(results_df)}")
results_df.head(20)

INFO - Use pytorch device_name: mps
INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5


'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 1015938c-1824-4370-9417-cedc2d75aff6)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-large-en-v1.5/resolve/main/./README.md


WARNING - '(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 1015938c-1824-4370-9417-cedc2d75aff6)')' thrown while requesting HEAD https://huggingface.co/BAAI/bge-large-en-v1.5/resolve/main/./README.md


Retrying in 1s [Retry 1/5].


WARNING - Retrying in 1s [Retry 1/5].


...

Using device: Apple Silicon GPU (MPS)
  macOS: True


Batches: 100%|██████████| 12/12 [00:00<00:00, 69.47it/s]

INFO - Refined 3 topics


INFO - Topic 0: Power Bank - Keywords: power bank, charger, charging, micro usb, device, 
INFO - Topic 1: Flashlight Battery Size - Keywords: battery, flashlight, built flashlight, charging, c
INFO - Topic 2: Battery and Charging Issues - Keywords: battery, looking battery, batteries, charger, rech

Total topics found: 8


,item,topic,reason,documents,num_documents
0,B008BA33Q4,Retro Silicone Case Design,"Keywords: case, cover, silicone, retro, cool, ...",[ like the desing so retro and it keeps my gal...,22
1,B008BA33Q4,Conversation Starter,"Keywords: conversation starter, conversation, ...",[ This cover Gets a lot of looks and sparks up...,4
2,B008BA33Q4,case fit,"Keywords: protect phone, cute case, way case, ...",[ fits the phone but now I have to put actuall...,16
3,B009AWT01Y,JBL Portable Speaker,"Keywords: little speaker, jbl, sound quality, ...",[ Bought this couple days ago and I am so far ...,23
4,B009AWT01Y,Bluetooth Speaker Sound Quality,"Keywords: bluetooth speaker, jbl flip, jbl, so...","[ I got mine at best buy,compared it on the sp...",24
5,B00AANQLRI,Power Bank,"Keywords: power bank, charger, charging, micro...",[ Bought this for my wife as we take frequent ...,20
6,B00AANQLRI,Flashlight Battery Size,"Keywords: battery, flashlight, built flashligh...",[ this thing even seems to be able to run my Q...,10
7,B00AANQLRI,Battery and Charging Issues,"Keywords: battery, looking battery, batteries,...",[ I took a chance on this battery and I am gla...,10


In [8]:
results_df.to_csv(project_root / "data/output" / "topic_modeling_results.csv", index=False)

In [9]:
# New cell to add after the model run

# Visualize aggregated results across all ASINs
from src.model.visualize import TopicVisualizer

# Create visualizer
vis = TopicVisualizer(template="plotly_white")

# 1. Topic Distribution across all ASINs
print("\n📊 Generating visualizations...")

# Count topics per ASIN
topics_per_asin = results_df.groupby('item').size()
fig_asin_topics = px.bar(
    x=topics_per_asin.index[:20],
    y=topics_per_asin.values[:20],
    labels={'x': 'ASIN', 'y': 'Number of Topics'},
    title='Topics per ASIN (Top 20)',
    template='plotly_white',
    color=topics_per_asin.values[:20],
    color_continuous_scale='Blues'
)
fig_asin_topics.update_layout(width=1000, height=500, xaxis_tickangle=-45)
fig_asin_topics.show()

# 2. Document distribution across topics
doc_counts = results_df.groupby('topic')['num_documents'].sum().sort_values(ascending=False)
fig_topic_dist = px.bar(
    x=doc_counts.index[:25],
    y=doc_counts.values[:25],
    labels={'x': 'Topic Name', 'y': 'Total Documents'},
    title='Document Distribution across Topics (Top 25)',
    template='plotly_white',
    color=doc_counts.values[:25],
    color_continuous_scale='Viridis'
)
fig_topic_dist.update_layout(width=1200, height=600, xaxis_tickangle=-45)
fig_topic_dist.show()

# 3. Topic frequency (how many ASINs have each topic)
topic_frequency = results_df.groupby('topic')['item'].nunique().sort_values(ascending=False)
fig_topic_freq = px.bar(
    x=topic_frequency.index[:20],
    y=topic_frequency.values[:20],
    labels={'x': 'Topic Name', 'y': 'Number of ASINs'},
    title='Topic Frequency across ASINs (Top 20)',
    template='plotly_white',
    color=topic_frequency.values[:20],
    color_continuous_scale='Reds'
)
fig_topic_freq.update_layout(width=1200, height=600, xaxis_tickangle=-45)
fig_topic_freq.show()

# 4. Heatmap: Topics vs ASINs (for top topics and ASINs)
top_topics = topic_frequency.index[:15].tolist()
top_asins_viz = topics_per_asin.index[:15].tolist()

pivot_data = results_df[results_df['topic'].isin(top_topics) & results_df['item'].isin(top_asins_viz)]
heatmap_matrix = pivot_data.pivot_table(
    index='topic', 
    columns='item', 
    values='num_documents', 
    aggfunc='sum',
    fill_value=0
)

fig_heatmap = px.imshow(
    heatmap_matrix,
    labels=dict(x="ASIN", y="Topic", color="Documents"),
    title="Topic-ASIN Heatmap (Top 15 each)",
    template='plotly_white',
    color_continuous_scale='YlOrRd',
    aspect='auto'
)
fig_heatmap.update_layout(width=1200, height=700)
fig_heatmap.show()

# 5. Detailed view: Sample topic details
print("\n📝 Sample Topic Details:")
print("="*80)
sample_topics = results_df.head(5)[['item', 'topic', 'reason', 'num_documents']]
for idx, row in sample_topics.iterrows():
    print(f"\nASIN: {row['asin']}")
    print(f"Topic: {row['topic']}")
    print(f"Reason: {row['reason']}")
    print(f"Documents: {row['num_documents']}")
    print("-"*80)

# 6. Summary Statistics
print("\n📈 Summary Statistics:")
print("="*80)
print(f"Total ASINs processed: {results_df['item'].nunique()}")
print(f"Total unique topics found: {results_df['topic'].nunique()}")
print(f"Total topic-ASIN combinations: {len(results_df)}")
print(f"Average topics per ASIN: {topics_per_asin.mean():.2f}")
print(f"Average documents per topic: {results_df['num_documents'].mean():.2f}")
print(f"Most common topic: {topic_frequency.index[0]} ({topic_frequency.values[0]} ASINs)")
print("="*80)


📊 Generating visualizations...



📝 Sample Topic Details:


KeyError: 'asin'